### Imports

In [185]:
# 1: Data preparation and preprocessing
import os
import random
import numpy as np
from skimage.util import random_noise
from skimage import io, transform, util
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 2: CNN Architecture and Transfer Learning
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# 3: CNN Training
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# 4: Setting up hyperparameters and comparing learning outcomes
import matplotlib.pyplot as plt

### Constants 

In [186]:
data_dir = 'data'
data_train = 'data/train'

# 1. Data Preparation

### 1.1. Generate validation set

In [187]:
import os
from skimage import io, transform
from skimage.util import random_noise
import random

# Function to apply random 3D rotation, noise, and resizing
def preprocess_image(image_path, output_size=(64, 64)):
    # Load the image
    image = io.imread(image_path)
    
    # Random 3D rotation (we'll simulate this with 2D rotations for simplicity)
    angle = random.randint(-15, 15)
    rotated_image = transform.rotate(image, angle, mode='wrap')
    
    # Add random noise
    noisy_image = random_noise(rotated_image, mode='gaussian')
    
    # Resize the image
    resized_image = transform.resize(noisy_image, output_size)
    
    # Convert the image data type to uint8
    resized_image = (resized_image * 255).astype(np.uint8)
    
    return resized_image

# Get a list of image paths
image_paths = [os.path.join(data_train, filename) for filename in os.listdir(data_train) if filename.endswith('.png')]

# Randomly select 20 images for the validation set
selected_images = random.sample(image_paths, 20)

# Apply preprocessing to each selected image
preprocessed_images = [preprocess_image(image_path) for image_path in selected_images]

# You can now use 'preprocessed_images' as your validation set
# If you want to save these images, you can do so using the 'io.imsave' function from skimage
for i, image in enumerate(preprocessed_images):
    io.imsave(f'data/validation/image_{i+1}.png', image)


In [188]:
# Initialize the ImageDataGenerator for data augmentation and normalization
datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize the images
    rotation_range=20,  # Rotate the images by up to 20 degrees
    width_shift_range=0.2,  # Shift the images horizontally by up to 20% of their width
    height_shift_range=0.2,  # Shift the images vertically by up to 20% of their height
    shear_range=0.2,  # Shear the images by up to 20 degrees
    zoom_range=0.2,  # Zoom in or out on the images by up to 20%
    horizontal_flip=True,  # Flip the images horizontally
    fill_mode='nearest'  # Fill in any missing pixels after a transformation
)

print(os.getcwd())

# Load and preprocess the images
train_generator = datagen.flow_from_directory(
    'data/train',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary',
    shuffle=True
)

validation_generator = datagen.flow_from_directory(
    'data/validation',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary',
    shuffle=True
)

print(train_generator.filepaths)
print(validation_generator.filepaths)

k:\Github\PaintingsTraining
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
[]
[]


# Task 2: CNN Architecture and Transfer Learning


### 2.1. Develop three different architectures from scratch

In [189]:
def create_model_1():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # Change to 'softmax' for multiclass
    return model

def create_model_2():
    model = Sequential()
    model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(64, 64, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))  # Change to 'softmax' for multiclass
    return model

def create_model_3():
    model = Sequential()
    model.add(Conv2D(32, (5, 5), activation='relu', input_shape=(64, 64, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (5, 5), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # Change to 'softmax' for multiclass
    return model

# Save the models
model_1 = create_model_1()
model_1.save('models/model_1.h5')

model_2 = create_model_2()
model_2.save('models/model_2.h5')

model_3 = create_model_3()
model_3.save('models/model_3.h5')

### 2.2. Transfer learning for each model

In [190]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.applications import VGG16

# Load the VGG16 model without the top layer
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3))

# Function to add a custom top layer to the base model for transfer learning
def add_custom_layers(base_model):
    model = Sequential()
    model.add(base_model)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # Change to 'softmax' for multiclass
    return model

# Create new models with transfer learning
tl_model_1 = add_custom_layers(vgg_base)
tl_model_2 = add_custom_layers(vgg_base)
tl_model_3 = add_custom_layers(vgg_base)

# Save the transfer learning models
tl_model_1.save('models/tl_model_1.keras')
tl_model_2.save('models/tl_model_2.keras')
tl_model_3.save('models/tl_model_3.keras')

# Task 3: CNN Training

In [191]:
def train_model(model, train_gen, val_gen, epochs=50):
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
    
    # Early stopping to prevent overfitting
    early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    
    # Model checkpoint to save the best model
    checkpoint = ModelCheckpoint(f'models/best_{model.name}.keras', save_best_only=True)
    
    history = model.fit(
        train_gen,
        epochs=epochs,
        validation_data=val_gen,
        callbacks=[early_stop, checkpoint]
    )
    
    return history


print(train_generator.filepaths)
print(validation_generator.filepaths)

# Train each model and save the history
history_1 = train_model(tl_model_1, train_generator, validation_generator)
history_2 = train_model(tl_model_2, train_generator, validation_generator)
history_3 = train_model(tl_model_3, train_generator, validation_generator)

[]
[]


ValueError: Asked to retrieve element 0, but the Sequence has length 0

# Task 4: Hyperparameters and Model Selection

In [ ]:
def plot_history(histories, titles):
    plt.figure(figsize=(15, 5))
    for i, history in enumerate(histories):
        plt.subplot(1, len(histories), i+1)
        plt.plot(history.history['accuracy'], label='Training Accuracy')
        plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
        plt.title(titles[i])
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.legend()
    plt.show()

# Plot the training and validation accuracies
plot_history([history_1, history_2, history_3], ['Model 1', 'Model 2', 'Model 3'])